In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import logging
import warnings
from pathlib import Path 

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import lightgbm as lgb 
import xgboost as xgb 
from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_auc_score

# 设置显示全部列
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')

### 数据预操作

In [3]:
data = pd.read_feather('data/data_for_model_v2.feather')
logging.info(f"data shape: {data.shape}")
data.head()

2021-01-26 10:22:51,408 INFO: data shape: (1000000, 77)


,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,years_between_issueDate_and_earliesCreditLine,issueDate_to_now_days,earliesCreditLine_to_now_days,ficoRange,revolBal_p_totalAcc,totalAcc_m_openAcc,loanAmnt_term,annualIncome_loanAmnt_term,debt,pro,installment_annualIncome,loanAmnt_applicationType,eny_num,int_sub,pro_dti,top1,rec,rec_rate,n_feat_min,n_feat_max,n_feat_sum,n_feat_mean,n_feat_median,n_feat_skew,n_feat_std,n_feat_mode,n_feat_range,n_feat_Q25,n_feat_Q75,sample
0,0,10.460938,5,3.021484,6.824219,4,21,5.769531,2.0,2,11.609375,2,2014-07-01,1.0,1,4.925781,3.496094,2.892578,0.0,730.0,734.0,2.080078,0.0,0.0,10.093750,3.910156,3.332031,0,0,Aug-2001,0.693359,1.0,0.0,2.0,2.0,2.0,4.0,2.302734,2.197266,1.609375,2.564453,2.0,2.080078,0.0,0.0,0.0,2.0,2.638672,7.785156,8.867188,4.0,10.093750,3.044922,8.851562,2.816406,14.445312,4.589844,0.008311,10.460938,6.308594,0.657273,1.905734,14.382812,0.0,0.0,0.0,12.0,52.0,3.714844,2.0,0.951172,3.851562,1.000000,12.0,0.50,6.25,train
1,1,9.796875,5,2.970703,6.136719,3,16,12.296875,5.0,0,10.734375,2,2012-08-01,0.0,0,5.054688,2.945312,3.361328,0.0,700.0,704.0,2.638672,0.0,0.0,9.625000,3.685547,2.945312,1,0,May-2002,7.453125,1.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,2.638672,NaN,NaN,NaN,NaN,2.398438,8.039062,8.828125,4.0,9.625000,1.791992,8.187500,2.623047,14.062500,4.539062,0.009991,9.796875,6.531250,0.768081,1.463708,14.593750,0.0,0.0,10.0,13.0,23.0,11.500000,11.5,NaN,2.121094,11.500000,3.0,10.75,12.25,train
2,2,9.390625,5,2.890625,5.699219,3,17,10.367188,8.0,0,11.210938,2,2015-10-01,0.0,0,5.824219,2.708984,3.167969,0.0,675.0,679.0,2.484375,0.0,0.0,8.437500,3.966797,3.332031,0,0,May-2006,0.000000,1.0,0.0,0.0,3.0,3.0,0.0,0.000000,3.091797,1.609375,1.791992,3.0,2.484375,0.0,0.0,0.0,4.0,2.302734,7.574219,8.593750,4.0,8.437500,2.833984,7.785156,3.460938,14.335938,4.453125,0.004021,9.390625,6.394531,0.692853,1.554091,14.093750,0.0,0.0,0.0,21.0,51.0,3.642578,1.5,2.322266,5.890625,0.000000,21.0,0.00,4.00,train
3,3,9.304688,3,2.111328,5.835938,0,3,10.757812,10.0,1,11.679688,1,2015-08-01,0.0,4,5.003906,2.484375,2.902344,0.0,685.0,689.0,2.302734,0.0,0.0,9.203125,3.982422,3.367188,1,0,May-1999,1.609375,1.0,6.0,4.0,6.0,6.0,4.0,2.833984,1.609375,2.080078,3.091797,6.0,2.302734,0.0,0.0,0.0,1.0,2.833984,7.605469,8.976562,4.0,9.210938,2.996094,8.210938,3.501953,14.523438,3.125000,0.002885,9.304688,6.210938,1.229641,0.817815,14.140625,0.0,0.0,0.0,21.0,84.0,6.000000,5.0,1.442383,6.078125,3.333984,21.0,1.75,6.75,train
4,4,8.007812,3,2.638672,4.625000,2,11,4.007812,NaN,1,10.273438,2,2016-03-01,0.0,10,5.710938,3.091797,3.501953,0.0,690.0,694.0,2.564453,0.0,0.0,7.988281,3.496094,3.332031,0,0,Aug-1977,2.484375,1.0,1.0,2.0,7.0,7.0,2.0,1.609375,2.302734,2.398438,2.773438,7.0,2.564453,0.0,0.0,0.0,4.0,3.689453,7.492188,9.671875,4.0,7.996094,2.773438,6.910156,3.400391,13.742188,3.687500,0.003479,8.007812,6.011719,0.779742,0.793786,14.031250,0.0,0.0,0.0,15.0,73.0,5.214844,4.0,0.664062,4.871094,0.000000,15.0,1.25,8.50,train


In [4]:
#训练数据/测试数据

features_to_drop = ['id','isDefault', 'sample']

X_train = data.loc[data['sample'] == 'train', :].drop(features_to_drop, axis=1)
X_test = data.loc[data['sample'] == 'test', :].drop(features_to_drop, axis=1)
y_train = data.loc[data['sample'] == 'train', 'isDefault']

X_train.shape, X_test.shape, y_train.shape

((800000, 74), (200000, 74), (800000,))

### 模型融合

In [5]:
def lgb_model(X_train, y_train, X_test, y_test=None):
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2)
    train_matrix = lgb.Dataset(X_train_split, label=y_train_split)
    valid_matrix = lgb.Dataset(X_val, label=y_val)

    params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.01,
        'min_child_weight': 0.32,
        'num_leaves': 14,
        'max_depth': 4,
        'feature_fraction': 0.81,
        'bagging_fraction': 0.61,
        'bagging_freq': 9,
        'min_data_in_leaf': 13,
        'min_split_gain': 0.27,
        'reg_alpha': 9.58,
        'reg_lambda': 4.62,
        'seed': 2020,
        'n_jobs':-1,
        'silent': True,
        'verbose': -1,
    }
    
    model = lgb.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], verbose_eval=500, early_stopping_rounds=200)
    # 计算在验证集上的得分
    val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    fpr, tpr, threshold = metrics.roc_curve(y_val, val_pred)
    roc_auc = metrics.auc(fpr, tpr)
    print('调参后lightgbm单模型在验证集上的AUC：{}'.format(roc_auc))

    # 对测试集进行预测
    test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    return test_pred

def xgb_model(X_train, y_train, X_test, y_test=None):
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2)
    train_matrix = xgb.DMatrix(X_train_split, label=y_train_split)
    valid_matrix = xgb.DMatrix(X_val, label=y_val)
    test_matrix = xgb.DMatrix(X_test)

    params = {
        'booster': 'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'gamma': 1,
        'min_child_weight': 1.5,
        'max_depth': 5,
        'lambda': 10,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'colsample_bylevel': 0.7,
        'eta': 0.04,
        'tree_method': 'exact',
        'seed': 2020,
        'n_jobs': -1,
        "silent": True,
    }

    watchlist = [(train_matrix, 'train'), (valid_matrix, 'eval')]
    model = xgb.train(params, train_matrix, num_boost_round=50000, evals=watchlist, verbose_eval=200, early_stopping_rounds=200)
    val_pred = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
    fpr, tpr, threshold = metrics.roc_curve(y_val, val_pred)
    roc_auc = metrics.auc(fpr, tpr)
    print('调参后xgboost单模型在验证集上的AUC：{}'.format(roc_auc))

    # 对测试集进行预测
    test_pred = model.predict(test_matrix, ntree_limit=model.best_ntree_limit)

    return test_pred

def cat_model(X_train, y_train, X_test, y_test=None):
    X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2)
    model = CatBoostClassifier(
        loss_function='Logloss',
        eval_metric='AUC',
        task_type='CPU',
        learning_rate=0.1,
        iterations=500,
        random_seed=2020,
        od_type='Iter',
        depth=7
    )
    model.fit(X_train_split, y_train_split, eval_set=(X_val, y_val), verbose=500, cat_features=col)
    val_pred = model.predict(X_val)
    fpr, tpr, threshold = metrics.roc_curve(y_val, val_pred)
    roc_auc = metrics.auc(fpr, tpr)
    print('调参后catboost单模型在验证集上的AUC：{}'.format(roc_auc))

    # 对测试集进行预测
    test_pred = model.predict(X_test, prediction_type='Probability')[:, -1]
    return test_pred

In [31]:
from heamy.dataset import Dataset
from heamy.estimator import Classifier

model_dataset = Dataset(X_train=X_train, y_train=y_train, X_test=X_test)
model_xgb = Classifier(dataset=model_dataset, estimator=xgb_model, name='xgb', use_cache=False)
model_lgb = Classifier(dataset=model_dataset, estimator=lgb_model, name='lgb', use_cache=False)
model_cat = Classifier(dataset=model_dataset, estimator=cat_model, name='cat', use_cache=False)

#### 使用Stacking方法进行模型融合

In [34]:
from heamy.pipeline import ModelsPipeline

pipeline = ModelsPipeline(model_cat, model_xgb, model_lgb)
pipeline

In [22]:
%%time 

# 构建第一层新特征， 其中K默认为5， 表示5折交叉验证， full_test=True, 对全部训练集进行训练得到基学习器，然后对测试集进行预测
stack_ds = pipeline.stack(k=5, seed=2020, full_test=True)

2021-01-16 15:06:49,228 INFO: Calculating xgb's fold #1
[15:06:50] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69992	eval-auc:0.69903
[200]	train-auc:0.73367	eval-auc:0.72818
[400]	train-auc:0.74162	eval-auc:0.73212
[600]	train-auc:0.74716	eval-auc:0.73379
[800]	train-auc:0.75177	eval-auc:0.73481
[1000]	train-auc:0.75591	eval-auc:0.73522
[1200]	train-auc:0.75970	eval-auc:0.73558
[1400]	train-auc:0.76335	eval-auc:0.73580
[1600]	train-auc:0.76683	eval-auc:0.73584
[1755]	train-auc:0.76938	eval-auc:0.73576
调参后xgboost单模型在验证集上的AUC：0.7358874704122116
2021-01-16 15:20:49,873 INFO: Calculating xgb's fold #2
[15:20:50] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be ac

In [23]:
# 第二层使用逻辑回归进行 stack
from sklearn.linear_model import LogisticRegression

stacker = Classifier(dataset=stack_ds, estimator=LogisticRegression, parameters={'solver': 'lbfgs'})
test_pred = stacker.predict()
test_pred

array([0.09313586, 0.35738364, 0.71145548, ..., 0.15995634, 0.18775493,
       0.07224607])

In [24]:
# 生成提交格式的DataFrame
df_result = pd.DataFrame({'id': data.loc[data['sample'] == 'test', 'id'].values, 'isDefault': test_pred})
df_result.sort_values(by='id').head(10)

,id,isDefault
0,800000,0.093136
1,800001,0.357384
2,800002,0.711455
3,800003,0.294181
4,800004,0.380386
5,800005,0.068497
6,800006,0.264772
7,800007,0.076216
8,800008,0.780822
9,800009,0.075935


In [25]:
df_result.to_csv('data/tc/pred_stacking.csv', index=False)

#### 使用blending方法进行模型融合

In [35]:
# 构建第一层新特征，将训练集切分为8：2
blend_ds = pipeline.blend(proportion=0.2, seed=111)
blender = Classifier(dataset=blend_ds, estimator=LogisticRegression, parameters={'solver': 'lbfgs'})
test_pred = blender.predict()
test_pred

NameError: name 'col' is not defined

In [27]:
# 生成提交格式的DataFrame
df_result = pd.DataFrame({'id': data.loc[data['sample'] == 'test', 'id'].values, 'isDefault': test_pred})
df_result.sort_values(by='id').head(10)

,id,isDefault
0,800000,0.092102
1,800001,0.356927
2,800002,0.657772
3,800003,0.284317
4,800004,0.398475
5,800005,0.067065
6,800006,0.289313
7,800007,0.076159
8,800008,0.748041
9,800009,0.075634


In [28]:
df_result.to_csv('data/tc/pred_blending_3.csv', index=False)